# 1. parsing data from AMAZON website.

import

In [1]:
from pattern import web
import urllib2
import json
import gzip
import pandas as pd
import requests
import json
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

from sklearn import linear_model
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import MultinomialNB

from sklearn.cross_validation import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cross_validation import train_test_split

fetching and get.. but become useless

In [2]:
def get_product_detail_page(asin):
    print "fetching " , asin
       
        
    #url = 'http://www.amazon.com/gp/product/' + (asin) + '/'
    url = 'http://rads.stackoverflow.com/amzn/click/' + (asin) + '/'
    
    page = requests.get(url)
    html = page.text
    """

    opener = urllib2.build_opener()
    opener.addheaders = [('User-agent', 'Mozilla/5.0')]
    response = opener.open(url)
    html = response.read()
    """

    return html

def parsing_detail_page(html):

    element= web.Element(html)
    ratele = element('span#acrPopover')
    if(ratele):
        ratedata = ratele[0].attr['title']
        return ratedata[:-15]
    else:
        print "fail!"
        return None

def fetch_rating(asin):
    return parsing_detail_page(get_product_detail_page(asin))


# 2.Load Review Data, and make list of products that have many reviews.

In [2]:
def parse(path):
    g = gzip.open(path, 'r')
    for l in g:
        yield eval(l)
        
def parse_only_many_reviews(path):
    """
    
    make dictionary
    { asin : ~~, review : {dd, dd, dd}, ratings : {r,r,r}}
    
    """
    pre_asin = ""
    done_asin = ""
    count = 0
    res_list = []
    res_dict = dict()
    review_list = []
    rating_list = []
    for review in parse(path):
        #same asin. count +1
        if(pre_asin == review['asin']):
            count = count+1
        else:
            if (count > 20):
                res_dict['asin'] = pre_asin
                res_dict['review'] = review_list
                res_dict['ratings'] = rating_list
                res_list.append(res_dict)
                res_dict = dict()
            review_list = []
            rating_list = []
            count = 1
            pre_asin = review['asin']
        review_list.append(review['reviewText'])
        rating_list.append(review['overall'])
    return res_list


def sum_and_rating(path, limit = False):
    """
    only use review which has many helpful.
    """
    res_dict = dict()
    res_list = []
    if(limit):
        for review in parse(path):
            helpful = review['helpful']
            if(helpful[1]>100):
                res_dict['review'] = review['reviewText']
                res_dict['rating'] = review['overall']
                res_list.append(res_dict)
                res_dict=dict()
    else:
        for review in parse(path):
            helpful = review['helpful']
            res_dict['review'] = review['summary']
            res_dict['rating'] = review['overall']
            res_list.append(res_dict)
            res_dict=dict()
    return res_list
        

def write_data(data,path):
    with open(path, 'w') as outfile:
        json.dump(data, outfile)
            

In [14]:
path = 'reviews_Apps_for_Android.json.gz'
data = parse_only_many_reviews(path)
write_data(data,'result_Android.json')

In [4]:
path = 'reviews_Apps_for_Android.json.gz'
data = sum_and_rating(path)
write_data(data,'Android_summary_each.json')
data = sum_and_rating(path,True)
write_data(data,'Android_summary_each_limit.json')

In [18]:
path = 'reviews_Apps_for_Android.json.gz'
data = sum_and_rating(path,True)
write_data(data,'Android_review_each_limit.json')

# Now, Read result file and make product_id,reviews,ratings data!

In [3]:
def read_data(path):
    with open(path) as data_file:    
        data = json.load(data_file)
    return data

def fetch_make_dataframe(data):
    """
    for parse_only_many_reviews
    """
    res = list()
    for i in range(0,len(data)):
        temp = dict()
        temp['asin'] = data[i]['asin']
        temp['review'] = data[i]['review']
        #parsing is failed, so I will use average data.
        temp['rating'] = sum(data[i]['ratings'])/len(data[i]['ratings']) #average.
        res.append(temp)
    return pd.DataFrame(res)

def fetch_mk_df_each(data):
    """
    for sum_and_rating
    """
    res = list()
    for i  in range(0,len(data)):
        temp = dict()
        temp['review'] = data[i]['review']
        temp['rating'] = data[i]['rating']
        res.append(temp)
    return pd.DataFrame(res)


In [21]:
path = 'result_Android.json'
data = read_data(path)
df = fetch_make_dataframe(data)
df

,asin,rating,review
0,B004A9SDD8,3.868852,[Glad to finally see this app on the android m...
1,B004AFQAUA,4.004329,[does not let you sample only for a sec. if yo...
2,B004AHBBPW,4.713528,[This app allows you to have access to daily d...
3,B004ALVL6W,2.735955,[if you have a bit of common sense this game h...
4,B004ANC00Q,3.421769,[This app simply latches on to Google maps and...
5,B004ANMWPY,3.684533,"[Easy easy easy!!! Love it!, joke, Love this a..."
6,B004AZH4C8,4.773946,"[I've liked sudoku for a few years now, and wo..."
7,B004AZSY4K,3.252336,[I wish I didn't download all these free app o...
8,B004BN3YQE,3.792308,[This was a free app of the day so I looked in...
9,B004C4FL5Y,4.338462,[I love this game! it is totally addictive!!!!...


In [14]:
path = 'Android_review_each_limit.json'
data = read_data(path)
df = fetch_mk_df_each(data)
df

,rating,review
0,5,My MP3 collection is gathering dust after gett...
1,5,"This app is great! It has a daily verse, a da..."
2,5,I love this app.I am one of those people that ...
3,5,it uses the battery up quickly so plug your ph...
4,5,On my HTC Thunderbolt it does not use much bat...
5,5,Does everything it says! Scans all different f...
6,1,"I used to love this app. I worked well, synced..."
7,1,"""Fixes a crash that some customers experienced..."
8,1,The newest update for Kindle takes up almost 3...
9,2,(12 of 12 people found this review helpful bef...


In [4]:
path = 'Android_summary_each_limit.json'
data = read_data(path)
df = fetch_mk_df_each(data)
df

,rating,review
0,5,World of music at your fingertips
1,5,Great resource with alot at your fingertips! (...
2,5,Awesome...
3,5,scanner
4,5,Excellent.. top quality of its type
5,5,Perfect
6,1,RECENT UPDATE KILLED ANDROID APP FOR ME
7,1,new update (3.5.1) hosed my android phone when...
8,1,Huge App
9,2,(v3.0.1.70) Great tablet app for Android (Moto...


# much better one.

# BOW

In [13]:
i=0
print df['review'][0][1]
df

o


,rating,review
0,5,World of music at your fingertips
1,5,Great resource with alot at your fingertips! (...
2,5,Awesome...
3,5,scanner
4,5,Excellent.. top quality of its type
5,5,Perfect
6,1,RECENT UPDATE KILLED ANDROID APP FOR ME
7,1,new update (3.5.1) hosed my android phone when...
8,1,Huge App
9,2,(v3.0.1.70) Great tablet app for Android (Moto...


In [6]:
import nltk
import numpy as np

In [7]:
def make_review_bow(data):    
   
    vectorizer = CountVectorizer(min_df=0)
    
    
    doc = []
    text1 =""
    for text in data['review']:
        doc.append(text)
    vectorizer.fit(doc)
    x = vectorizer.transform(doc)
    bow2 = x.toarray()
    
    bow = pd.DataFrame(bow2)

    return vectorizer, bow

In [24]:
def make_review_bow2(data):    
   
    vectorizer = CountVectorizer(min_df=0)
    
    
    doc = []
    text1 =""
    for te in data['review']:
        for text in te:
            doc.append(text)
    vectorizer.fit(doc)
    x = vectorizer.transform(doc)
    bow2 = x.toarray()
    
    bow = pd.DataFrame(bow2)

    return vectorizer, bow

In [ ]:
vectorizer, bow_X = make_review_bow2(df)
res = pd.DataFrame()
for attri in bow_X:
    if int(sum(bow_X[attri])) > 1:
        res[attri] = bow_X[attri]
Y = df['rating']
bow_X=res
train_X, test_X, train_Y, test_Y = train_test_split(bow_X, Y, test_size=0.2)

In [23]:

sgd = SGDClassifier()
sgd.fit(train_X, train_Y)
print "Model Accuracy: %f" % (100 * sgd.score(test_X, test_Y))
print "Model Accuracy: %f" % (100 * sgd.score(train_X, train_Y))

KeyboardInterrupt: 

In [17]:
#for all
sgd = SGDClassifier()
sgd.fit(train_X, train_Y)
print "Model Accuracy: %f" % (100 * sgd.score(test_X, test_Y))
print "Model Accuracy: %f" % (100 * sgd.score(train_X, train_Y))

Model Accuracy: 64.331984
Model Accuracy: 75.503593


In [18]:
bow_X #original -> 0~4736 feature
#after delete sum == 1 -> 2122 feature

,0,1,5,7,8,9,11,22,23,24,...,20546,20548,20549,20551,20552,20553,20554,20555,20556,20561
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [12]:
bow_X

,0,4,5,11,13,15,16,18,19,20,...,4715,4721,4723,4724,4726,4727,4731,4732,4734,4736
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
